In [78]:
import os
import pandas as pd
import numpy as np

In [79]:
data = pd.read_csv(
    "data/Food Ingredients and Recipe Dataset with Image Name Mapping.csv"
)
data.head()

,Unnamed: 0,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (..."
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."


In [80]:
# Cleanup

# drop all nan values
data = data.dropna()

In [81]:
from langchain.schema import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Qdrant

# embedding config - using All MiniLM L6 v2
os.environ["OPENAI_API_KEY"] = "random-string"
embeddings = OpenAIEmbeddings(openai_api_base="http://localhost:8444/v1")

In [95]:
# For testing qdrant as a standalone service - ignore

# from qdrant_client import QdrantClient
# client = QdrantClient("localhost", port=6333)

# client.get_collections()

# client.delete_collection(collection_name="new_recipe_collection")

# client.get_collections()

# Random Vector - An array of 384 random numbers
# random_vector = np.random.rand(384)

# search_result = client.search(
#     collection_name="new_recipe_collection", query_vector=random_vector, limit=3
# )
# print(search_result)

[ScoredPoint(id='a7439f9d-fa9b-4751-a897-0bac5f83687a', version=147, score=0.10512468, payload={'metadata': {'image': 'ratatouille-350730.jpg', 'recipe': 'Preheat oven to 350°F.\nIn large saucepan over moderately high heat, bring 2 quarts <epi:recipelink id="105591">salted water</epi:recipelink> to boil. Add fennel and boil until tender, about 10 minutes.\nMeanwhile, rub red and yellow peppers, zucchini, and eggplant with olive oil and transfer to large baking sheet. Cut off top 1/2 inch from garlic head. Wrap in foil and transfer to baking sheet alongside vegetables.\nWhen fennel is tender, use tongs to transfer to sheet and rub with oil. Roast vegetables, turning occasionally, until tender and slightly browned, 30 to 40 minutes.\nTransfer roasted peppers to large bowl and cover with plastic wrap. Let all vegetables stand until cooled slightly, about 10 minutes. Peel and deseed peppers, then finely dice. Remove seeds from zucchini and finely dice. Finely dice eggplant. Core fennel and

In [83]:
docs=[]

for index, row in data.iterrows():
    vector_embedding = embeddings.embed_query(row["Title"])
    docs.append(Document(
                page_content=row["Title"], metadata={"recipe": row["Instructions"], "image": f"{row['Image_Name']}.jpg"}
             ))
vectorstore = Qdrant.from_documents(
    docs,
    embeddings,
    url="http://localhost:6333",  # Qdrant gRPC API endpoint
    collection_name="new_recipe_collection",
)

100%|██████████| 1/1 [00:00<00:00,  4.98it/s]


In [96]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.chat_models import ChatOpenAI

# Defining metadata fields
metadata_field_info = [
    AttributeInfo(
        name="recipe",
        description="The recipe of the dish",
        type="string",
    ),
    AttributeInfo(
        name="image",
        description="The file name of the image",
        type="string",
    ),
]
document_content_description = "Title of the recipe"

# Chat completion llm - Vicuna 7B
llm = ChatOpenAI(
    openai_api_base="http://localhost:8222/v1", max_tokens=128, temperature=0.9
)

# llm = ChatOpenAI(openai_api_key="the_api_key")
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True
)

In [97]:
# This example only specifies a relevant query
retriever.get_relevant_documents("What are recipes with chicken where an oven is not used?")

/Users/aditya/anaconda3/envs/aditya/lib/python3.11/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query='chicken' filter=Operation(operator=<Operator.NOT: 'not'>, arguments=[Comparison(comparator=<Comparator.CONTAIN: 'contain'>, attribute='recipe', value='oven')]) limit=None


[Document(page_content='Fried Chicken', metadata={'image': '#NAME?.jpg', 'recipe': 'Place buttermilk in a large, shallow nonreactive dish and add chicken. Cover and refrigerate for at least 2 hours or overnight.\nCombine flour, paprika, cayenne, garlic powder, salt, and pepper in a large plastic bag.\nRemove chicken pieces from buttermilk and add to bag with flour mixture. Shake to coat; dust off excess.\nPour 3/4" peanut oil into a heavy skillet. Heat oil over medium-high heat until it registers 350° on a deep-fry thermometer. Reduce heat to medium and add the chicken pieces, a few at a time. Fry each piece for 10-12 minutes, turning occasionally, until it is crispy and golden and internal temperature registers 165° on an instant-read thermometer. Drain on paper towels; serve.'}),
 Document(page_content='Butter Chicken', metadata={'image': 'indian-style-butter-chicken-curry.jpg', 'recipe': 'Cut the chicken into 1-inch chunks and toss them with the lemon juice and chile powder. Transfe